In [268]:
import re
import hashlib
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, hstack

from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

In [269]:
path_to_train = "train.csv"
path_to_test = "test.csv"

In [270]:
dataset_train = pd.read_csv(path_to_train)
dataset_test = pd.read_csv(path_to_test)
dataset_train.shape, dataset_test.shape

((135309, 4), (165378, 3))

In [290]:
dataset_test.head(10)

,ID,url,title
0,135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...
1,135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ..."
2,135311,imperimeha.ru,Женщинам | Империя Меха - Part 12
3,135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ..."
4,135313,2gis.ru,67
5,135314,ryazan.gorodrabot.ru,"Вакансия Мерчендайзер визитный в Рязани, Групп..."
6,135315,m.fotosklad.ru,Смартфон Apple iPhone XR 64GB Черный A2105 EU ...
7,135316,sim-dealer.ru,Духовой шкаф Siemens HB337GYS0R - Sim-Dealer. ...
8,135317,krasnoyarsk.hh.ru,Вакансия Аналитик Департамента внедрения инфор...
9,135318,vsetop.org,Anomaly Defenders v1.0 – торрент


# 1. tf-idf для фичи title

Сначала попробуем оставить только title, т.к. он содержит в себе наибольшее количество информации. Векторизуем title при помощи tf-idf.

Предварительно уберём все строки, имеющие пустой title.

In [272]:
missing_rows = dataset_train[dataset_train['title'].isnull()]
print(missing_rows)

          ID        url title  label
78497  78497  jpg-1.com   NaN      0


In [273]:
dataset_train = dataset_train.dropna(subset=["title"])
x_train, x_test, y_train, y_test = train_test_split(dataset_train["title"], dataset_train["label"], 
                                                    test_size=0.2, random_state=42)

In [274]:
%%time
vectorizer = TfidfVectorizer()
x_train_vect = vectorizer.fit_transform(x_train)
x_test_vect = vectorizer.transform(x_test)

CPU times: user 1.27 s, sys: 55.3 ms, total: 1.33 s
Wall time: 1.33 s


In [275]:
(x_train_vect.shape, y_train.shape) #164 677 фич

((108246, 164677), (108246,))

In [276]:
clf = LogisticRegression(penalty="l2")

In [277]:
%%time
clf.fit(x_train_vect, y_train)
predicted = clf.predict(x_test_vect)
print("Precision: %.3f" % precision_score(Y_test, predicted))
print("Recall: %.3f" % recall_score(Y_test, predicted))
print("F1: %.3f" % f1_score(Y_test, predicted))

Precision: 0.984
Recall: 0.852
F1: 0.914
CPU times: user 5.61 s, sys: 1.49 s, total: 7.1 s
Wall time: 1.03 s


Как видим, F1-score уже > 0.911, то есть такой простой подход уже показывает хороший результат

# 2. tf-idf для конкатенации title и url

Попробуем сконкатенировать title с url. Получим новую фичу, которая имеет большее количество "триггерных" токенов по сравнению с title.

Вернём в датасет удалённые строки

In [278]:
dataset_train = pd.concat([dataset_train, missing_rows])
dataset_train = dataset_train.sort_index()
dataset_train.shape

(135309, 4)

In [279]:
dataset_train["title_url"] = np.where(
    dataset_train["title"].isnull(), 
    dataset_train["url"], 
    dataset_train["title"] + " " + dataset_train["url"]
)
x_train, x_test, y_train, y_test = train_test_split(dataset_train["title_url"], dataset_train["label"], 
                                                    test_size=0.2, random_state=42)

In [280]:
x_train.head(10)

36898     Амиксидин: продажа, цена в Санкт-Петербурге. м...
69667                        Ультрасет new samara.farfor.ru
111124    Lada Vesta Cross: видеообзор с презентации сер...
35818     Фрезерный станок Моделист CNC-6090AL 600х900мм...
123149    Популярное порно по просмотрам - порнуха попул...
108474    Вакансия Водитель-экспедитор (м. Динамо) в Дол...
52553     Сообщишь ударение, как правильно пишется слово...
46513     Методика обучения чтению и русскому языку в на...
7328      Фальшивые кумиры окраин — Новые Русские Новост...
130863    Photoshop CC для начинающих 2017 Роберт Шаффлб...
Name: title_url, dtype: object

In [281]:
%%time
hashing_vectorizer_sklearn = HashingVectorizer(n_features=350000)
x_train_vect = hashing_vectorizer_sklearn.fit_transform(x_train)
x_test_vect = hashing_vectorizer_sklearn.transform(x_test)

CPU times: user 978 ms, sys: 14.8 ms, total: 992 ms
Wall time: 997 ms


In [282]:
clf.fit(x_train_vect, y_train)
predicted = clf.predict(x_test_vect)
print("Precision: %.3f" % precision_score(Y_test, predicted))
print("Recall: %.3f" % recall_score(Y_test, predicted))
print("F1: %.3f" % f1_score(Y_test, predicted))

Precision: 0.123
Recall: 0.108
F1: 0.115


In [ ]:
А начиналось так красиво! Значит необработанный url всё портит.

# 3. tf-idf для конкатенации title + список триггерных слов для url

Заведём словарь слов, связанных с порнографией. Сделаем из него регулярку, которую будем метчить с url.

In [283]:
triggers = {"porn", "xxx", "sex", "adult", "blowjob", "milf", "fuck", "dick", "pussy", "18+", "erotic", "masturbation",
"nude", "explicit", "rape", "cum", "condom", "orgasm", "hotwife", "genital", "nudie"}

pattern_str = "|".join(map(re.escape, triggers))
trigger_pattern = re.compile(pattern_str, flags=re.IGNORECASE)

In [284]:
dataset_train["trigger_url"] = dataset_train["url"].apply(lambda x: len(trigger_pattern.findall(x)))
whoops_rows = dataset_train[dataset_train["trigger_url"] != 0]
print(whoops_rows)

            ID                url  \
12          12      pornmult.info   
52          52    desixxxtube.pro   
55          55        daftsex.com   
57          57   dohera-porno.net   
111        111        daftsex.com   
...        ...                ...   
135214  135214   ruporno-seks.com   
135239  135239      pornotwix.com   
135253  135253         sexlib.org   
135256  135256  pakistanporn.info   
135287  135287         sexlib.org   

                                                    title  label  \
12      кримпай,мать и сын » Страница 5 » смотреть пор...      1   
52      Indian aunty bjowjob and fucking with her part...      1   
55                             Playlist Lesbian — DaftSex      1   
57      Порно видео №309 - эротические фильмы с малоле...      1   
111                            Charlie Red Anal – DaftSex      1   
...                                                   ...    ...   
135214  Привлекательная худышка залезла на белый диван...      1   
135239     

Видно, что url, содержащих слова с порнографией, не так уж и мало. 

In [285]:
dataset_train = dataset_train.dropna(subset=["title"])
x_train, x_test, y_train, y_test = train_test_split(dataset_train[["title", "trigger_url"]], dataset_train["label"], 
                                                    test_size=0.2, random_state=42)

Сначала векторизуем с помощью tf-idf title как в пункте 1. А затем добавим столбец с бинарным признаком trigger_url, который показывает, встретилось ли триггерное слово в url или нет.

In [286]:
%%time
vectorizer = TfidfVectorizer()
x_train_title_vect = vectorizer.fit_transform(x_train["title"])
x_test_title_vect = vectorizer.transform(x_test["title"])

x_train_trigger_vect = csr_matrix(x_train["trigger_url"].values.reshape(-1, 1))
x_test_trigger_vect = csr_matrix(x_test["trigger_url"].values.reshape(-1, 1))

x_train_vect = hstack([x_train_title_vect, x_train_trigger_vect])
x_test_vect = hstack([x_test_title_vect, x_test_trigger_vect])

CPU times: user 1.26 s, sys: 51.2 ms, total: 1.31 s
Wall time: 1.31 s


In [287]:
(x_train_vect.shape, y_train.shape)

((108246, 164678), (108246,))

In [288]:
clf.fit(x_train_vect, y_train)
predicted = clf.predict(x_test_vect)
print("Precision: %.3f" % precision_score(Y_test, predicted))
print("Recall: %.3f" % recall_score(Y_test, predicted))
print("F1: %.3f" % f1_score(Y_test, predicted))

Precision: 0.991
Recall: 0.896
F1: 0.941


F1-score улучшился на 3%. Исходя из этого, выберем именно эту модель для предсказания на тестовой выборке из kaggle.

# 4. Итоговое решение и сохранение результата в sample_submission.csv


In [293]:
dataset_test["trigger_url"] = dataset_test["url"].apply(lambda x: len(trigger_pattern.findall(x)))

x_train, y_train = dataset_train[["title", "trigger_url"]], dataset_train["label"]
x_test = dataset_test[["title", "trigger_url"]]

In [295]:
%%time
vectorizer = TfidfVectorizer()
x_train_title_vect = vectorizer.fit_transform(x_train["title"])
x_test_title_vect = vectorizer.transform(x_test["title"])

x_train_trigger_vect = csr_matrix(x_train["trigger_url"].values.reshape(-1, 1))
x_test_trigger_vect = csr_matrix(x_test["trigger_url"].values.reshape(-1, 1))

x_train_vect = hstack([x_train_title_vect, x_train_trigger_vect])
x_test_vect = hstack([x_test_title_vect, x_test_trigger_vect])

(x_train_vect.shape, y_train.shape)

CPU times: user 2.42 s, sys: 56.2 ms, total: 2.48 s
Wall time: 2.48 s


((135308, 189853), (135308,))

In [298]:
clf.fit(x_train_vect, y_train)
predicted = clf.predict(x_test_vect)

In [300]:
submission = pd.DataFrame({
    'ID': dataset_test['ID'],
    'label': predicted
})

submission.to_csv('sample_submission.csv', index=False)